In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import healpy as hp
import numpy as np
import h5py, os

from astropy.io import fits
from functools import reduce

from msfm.utils.input_output import read_yaml

### global constants

In [2]:
conf_dir = "../../configs/config.yaml"
conf = read_yaml(conf_dir)

n_side = conf["analysis"]["n_side"]
n_pix = conf["analysis"]["n_pix"]

pixels_file = f"../../data/DESY3_pixels_{n_side}.h5"
noise_file = f"../../data/DESY3_noise.h5"

# Metacal

In [3]:
# exclude the 0th bin, which includes all of the others
tomo_inds = [1, 2, 3, 4]
z_lims = conf["survey"]["metacal"]["z_lims"]
z_bins = conf["survey"]["metacal"]["z_bins"]

### load Metacal catalog

In [4]:
# data given by Dominik
data_dir = '/Users/arne/data/DESY3/DES_Y3KP_NGSF/'

e1_tomo = []
e2_tomo = []
w_tomo = []
# consider all tomographic bins
for tomo in tomo_inds:
    # ellipticities for a fixed tomographic bin, these are doubles
    e1 = h5py.File(os.path.join(data_dir, f'cal_e1_tomo={tomo}.h5'))['cal_e1'][:]
    e2 = h5py.File(os.path.join(data_dir, f'cal_e2_tomo={tomo}.h5'))['cal_e2'][:]
    w = h5py.File(os.path.join(data_dir, f'weight_tomo={tomo}.h5'))['weight'][:]

    assert e1.shape == e2.shape == w.shape

    print(f"tomo bin {tomo} contains {len(e1)} galaxies")
    e1_tomo.append(e1)
    e2_tomo.append(e2)
    w_tomo.append(w)

tomo bin 1 contains 24940465 galaxies
tomo bin 2 contains 25280405 galaxies
tomo bin 3 contains 24891859 galaxies
tomo bin 4 contains 25091297 galaxies


# checks

In [5]:
# check that we have zero means
for tomo, e1, e2, w in zip(tomo_inds, e1_tomo, e2_tomo, w_tomo):
    # calculate
    print("Z bin: ", tomo)
    e1_mean = np.sum(e1*w)/np.sum(w)
    e2_mean = np.sum(e2*w)/np.sum(w)
    print("Mean e1: ", e1_mean)
    print("Mean e2: ", e2_mean)

Z bin:  1
Mean e1:  -1.0454848110573765e-18
Mean e2:  -2.2662481407338397e-18
Z bin:  2
Mean e1:  1.9298109539069204e-18
Mean e2:  4.4170040337157934e-18
Z bin:  3
Mean e1:  2.3564963418725306e-18
Mean e2:  -1.0293771723659463e-18
Z bin:  4
Mean e1:  1.65857831039232e-18
Mean e2:  8.000530525804698e-18


# create the noise file

In [6]:
with h5py.File(noise_file, "w") as f:
    for tomo, z_bin, e1, e2, w in zip(tomo_inds, z_bins, e1_tomo, e2_tomo, w_tomo):
        # shape (n_gals, 3)
        sample = np.stack([e1, e2, w], axis=1)

        # save as np.float32, single precision
        dset = f.create_dataset(name=z_bin, data=sample, dtype="f")
        dset.attrs["info"] = "This dataset contains the ellipticities of the Metacalibration galaxy sample per redshift bin. " \
                             "The shape is (n_gals, 3), where axis 1 contains (e1, e2, w)."
